<a href="https://colab.research.google.com/github/pjvperes/DAO-using-JS/blob/main/Desempenho_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Integrantes:

Pedro Henrique Valvezon Maldonado, N° USP: 11797240

Giovanni Jasper Trigueros, N° USP:11918310

Jean Carlos Matter Thomasio, N° USP:10820870

Pedro Nascimento Campos Peres, N° USP:11797208

Arsène Bokani Costa, N° USP:11737926

Victor Dias Teixeira, N° USP:11232966

In [ ]:
!pip install ambiance

import ambiance as amb
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere


# Questão:
Considere os dados da seguinte aeronave:

In [ ]:
dados_aeronave = {
    "AR": 8.90,
    "e": 0.85,
    "CD0": 0.025,
    "MTOW": 450_300, # N
    "W_fuel": 130_000, # N
    "thrust": 92300, # N
    "A": 92.5, # m**2
    "TSFC": 0.85/3600, # N/s/N
    "BOW": 320_300 # N peso da aeronave sem combustivel
}

condicoes = {
    "H_partida": 0,
    "ISA_destino": 15 , # ISA + 15
    "H_max": 41_000, #ft
    "H_final": 0,
    "d_alternativa": 200, #nm
    "H_cruise": 20_000, # ft
    "ISA_alternativa" : 20, # ISA+20
    "H_loiter": 10_000, #ft
    "dT_loiter": 30, # min
    "H_alternativa": 2000, # ft
}

regime_long_range = {
    "M": 0.79,
    "RS_min": 500, # ft/min
    "dH_step_climb": 2000
}



O alcance deverá ser calculado utilizando o regime ”long range” e com mach
constante de 0.79;

Caso haja necessidade de se efetuar um ”step climb” (Razãoo de subida menor
que 500 ft/min), essa deverá ser feita em um nível de voo adequado, ou seja, a
cada 2000 ft. Exemplo, considerando o FL 410, o step climb deve ser feito no
FL 390, nunca no FL 400.

Por fim, como essa aeronave se compara com outras de sua categoria.

## Subida

Implementação dos trechos de subida.

In [ ]:
def subida(dados_aeronave, h0, h_final):
    print('h0 = ', h0)
    # Deve retornar o tempo decorrido, combustivel consumido, distância percorida e variação da altura
    ## Dados da aeronave
    AR=dados_aeronave["AR"]
    e=dados_aeronave["e"]
    Cd0 = dados_aeronave["CD0"]
    BOW = dados_aeronave["BOW"] # N
    W_fuel = dados_aeronave['W_fuel'] # N
    Thrust0 = dados_aeronave["thrust"] # N
    S = dados_aeronave["A"]
    TSFC = dados_aeronave["TSFC"]
    K = 1/(np.pi*AR*e)

    # Condições atmosféricas
    T0 = Atmosphere(0).temperature[0]
    a0 = Atmosphere(0).speed_of_sound[0]
    P0 =  Atmosphere(0).pressure[0]
    rho0 =  Atmosphere(0).density[0]

    ## Iniciação das variaveis
    t = 0
    tempo_total = 0
    consumo_total = 0
    dist_percorrida = 0
    W = BOW + W_fuel

    # Chutes do intervalo de velocidade a ser avaliado
    vmin = 100
    vmax = 300

    # Cria um range de velocidades a serem avaliadas
    V = np.arange(vmin, vmax, 100)

    # Variação da temperatura ISA
    dT = 0

    # variáveis auxiliares
    consumos = []
    distancias = []
    tempos = []
    RCs_max = []

    discretização = 100                                                         # intervalos de altitudes
    # realizando diversos cálculos para cada altura dentro do range de 0 à 41000 pés
    alturas = np.arange(h0, h_final+discretização, discretização)

    #print('alturas = ', alturas)
    for h_ind in alturas:
        # Corrige a variação de temperatura ISA
        T      = Atmosphere(h_ind * 0.3048).temperature[0]                      # altitude em metros
        h_real = h_ind * (T + dT)/T

        # Calculo
        sigma = (Atmosphere(h_real * 0.3048).density)[0]/rho0
        M     = (V)/Atmosphere(h_real * 0.3048).speed_of_sound[0]               # V (true aero speed)  (V*sigma**0.5)= Ve (Velocidade equivalente)
        phi   = 1/(0.7 * M**2)  *  (((1 + 0.2 * M**2)**3.5 - 1)/((1 + 0.2 * M**2)**2.5))
        f_a   =  0.7 * (M**2) * (phi-(0.190263 * (T+dT)/T))

        # Variação do empuxo
        Thrust = Thrust0 * sigma**0.7                                           # empuxo para menor q troposfera
        if h_real > 36089:
            Thrust = 1.439 * Thrust0 * sigma                                    # empuxo para altitudes maiores q troposfera
            f_a    = phi * 0.7 * M**2                                           # fator de aceleração para altitudes maiores q troposfera

        ## Calcula Consumo
        consumo = TSFC * Thrust * t
        consumos.append(consumo)
        W = W - consumo

        Cl = (2 * (W/S))/( Atmosphere(h_real*0.3048).density * V**2)

        Cd = Cd0 + K * (Cl**2)

        E = Cl/Cd

        gamma = np.arcsin(Thrust/W - 1/E)

        # Calculo da razão de subida máxima
        RC_1 = V * np.sin(gamma)
        RC = RC_1/(1 + f_a)
        RCs_max.append(max(RC))

        index = np.where(RC == max(RC))                                         # pega o index do vetor RC em que RC é máximo
        v_rcmax = V[index][0]                                                   # pega a velocidade em que RC é máximo
        gamma_rcmax = gamma[index][0]                                           # pega o gama em que RC é máximo
        v_horizontal = v_rcmax * np.cos(gamma_rcmax)                            # calcula a velocidade horizontal em que RC é máximo para determinada faixa de altitude


        t = discretização * ((T + dT)/T) * 0.3048 / max(RC)                     # Tempo para o maximo RC de cada altitude = faixa de altitude/RC
        tempos.append(t)
        d = v_horizontal * t                                                    # Distância percorrida
        distancias.append(d)


    if int(h_final/1000)%2 == 1:
        flight_levels = np.arange(1000, h_final + 1000, 2000)                     # Array com todos os FL até o max
    else:
        flight_levels = np.arange(0, h_final + 2000, 2000)
    mask = np.array(RCs_max)<2.54                                               # Mask p/ alturas que precisam de stepclimb
    #print('mask = ', mask)
    if flight_levels[flight_levels > h0].size == 0:
      FL = h0
    else:
      flight_levels = flight_levels[flight_levels>=h0]
      if alturas[mask].size == 0:
          FL = max(flight_levels[flight_levels>h0])
      else:
          min_alt_step_climb = min(alturas[mask])
          if flight_levels[flight_levels < min_alt_step_climb].size == 0:                                            # Min h p/ step climb
            FL = h0
          else:
            FL = max(flight_levels[flight_levels < min_alt_step_climb])
    #print('flight_levels[flight_levels < min_alt_step_climb] = ', flight_levels[flight_levels < min_alt_step_climb])
    mask2 = alturas < FL                                                        # Itens que estão a alturas menores que o flight level

    tempo_total     = sum(np.array(tempos)[mask2])
    dist_percorrida = sum(np.array(distancias)[mask2])
    consumo_total   = sum(np.array(consumos)[mask2])

    #print(f'{tempo_total/60:.3f} min, {consumo_total } , {dist_percorrida/1000:.3f}km, {FL} ft')

    return tempo_total, consumo_total, dist_percorrida, FL

#print('Subida 1 p/ cruseiro')
_, consumo_total_subida_1, _, FL_1 = subida(dados_aeronave, condicoes['H_partida'], condicoes['H_max'])



h0 =  0


## Cruzeiro

Implementação do trecho de cruzeiro

In [ ]:
def cruzeiro(dados_aeronave, condicoes, FL):
    # Deve retornar o tempo decorrido, combustivel consumido, distância percorida e variação da altura
    ## Dados da aeronave
    AR= dados_aeronave["AR"]
    BOW= dados_aeronave["BOW"]
    e = dados_aeronave["e"]
    Cd0 = dados_aeronave["CD0"]
    MTOW = dados_aeronave["MTOW"] # N
    Thrust0 = dados_aeronave["thrust"] # N
    S = dados_aeronave["A"]
    tsfc = dados_aeronave["TSFC"]
    w_fuel = dados_aeronave['W_fuel']
    mach = 0.79                                                                 #nem vai precisar usar 99% da vel ja q o mach é contante
    K = 1/(np.pi*AR*e)
    # Condições atmosféricas
    isa = condicoes["ISA_destino"]
    #altitude = condicoes["H_partida"] * 0.3048 # 41000 pés em metros

   # Condições atmosféricas
    isa = 0
    altitude = FL* 0.3048 # [m]
    temp = Atmosphere(altitude).temperature - isa                               # Dados atmosféricos da biblioteca Ambiance
    a = Atmosphere(altitude).speed_of_sound                                     # Dados atmosféricos da biblioteca Ambiance
    P = Atmosphere(altitude).pressure                                           # Dados atmosféricos da biblioteca Ambiance
    rho =  Atmosphere(altitude).density
    v = mach*a
    dt=100
    distancia_total=0
    dist=0


    # Peso total e de combustível ao se iniciar cruzeiro
    W = BOW + w_fuel                                                          # Peso total no inicio do cruzeiro
    tempo = list(range(0,2000000,dt))


    # Cd e eficiência no início do cruzeiro
    Cl = (2 * (W/S))/( Atmosphere(altitude).density * v**2)
    Cd_inicial = Cd0 + K * (Cl**2)
    E = Cl/Cd_inicial
    T = W/E



    distancia_cruzeiro_total=0 #distancia total percorrida inicial
    dist_cruzeiro=0
    controle = BOW + 31000 # precisa de 31000 N de combustivel pra fazer o resto do trajeto.
    t = 0
    while True:
       t+=dt
       v_operacional = mach*a
       Cl = (2 * (W/S))/( Atmosphere(altitude).density * v_operacional**2) #calculo do cl
       Cd = Cd0 + K * (Cl**2)
       E = Cl/Cd
       T = W/E
       W -= tsfc*dt*T
       dados_aeronave['W_fuel'] = dados_aeronave['W_fuel'] - tsfc*dt*T

       if  W < controle:
        print('A aeronave usou todo o combustível disponível para cruzeiro!\n')
        print('tempo total gasto em cruzeiro [min] = ', t/60)
        break

       tempo_total, consumo_total, dist_percorrida, FLi = subida(dados_aeronave, FL, 41000)
       print('FL = ', FLi, ' tempo = ', t, '\n')
       FL = FLi
       W -= consumo_total
       t += tempo_total

      # Calcula a distância percorrida
       dist = v_operacional * dt # distancia percorrida para cada faixa de tempo
       distancia_total = distancia_total + dist + dist_percorrida # soma de distancias percorridas para cada velocidade ideal

    print('ditancia total percorrida [km] =',distancia_total/1000)

    plt.show()





    print('ditancia total percorrida [km] =',distancia_total/1000)

dados_aeronave['W_fuel'] -= consumo_total_subida_1
cruzeiro(dados_aeronave, condicoes, FL_1)


h0 =  37000
FL =  37000  tempo =  100 

h0 =  37000
FL =  37000  tempo =  200 

h0 =  37000
FL =  37000  tempo =  300 

h0 =  37000
FL =  37000  tempo =  400 

h0 =  37000
FL =  37000  tempo =  500 

h0 =  37000
FL =  37000  tempo =  600 

h0 =  37000
FL =  37000  tempo =  700 

h0 =  37000
FL =  37000  tempo =  800 

h0 =  37000
FL =  39000  tempo =  900 

h0 =  39000
FL =  39000  tempo =  1196.5594247825106 

h0 =  39000
FL =  39000  tempo =  1296.5594247825106 

h0 =  39000
FL =  39000  tempo =  1396.5594247825106 

h0 =  39000
FL =  39000  tempo =  1496.5594247825106 

h0 =  39000
FL =  39000  tempo =  1596.5594247825106 

h0 =  39000
FL =  39000  tempo =  1696.5594247825106 

h0 =  39000
FL =  39000  tempo =  1796.5594247825106 

h0 =  39000
FL =  39000  tempo =  1896.5594247825106 

h0 =  39000
FL =  39000  tempo =  1996.5594247825106 

h0 =  39000
FL =  39000  tempo =  2096.5594247825106 

h0 =  39000
FL =  39000  tempo =  2196.5594247825106 

h0 =  39000
FL =  39000  tempo =  2

## Descida para destino 1

Implementação do trecho de decida

In [ ]:
import ambiance as amb
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere


def descida_destino1():
    AR=dados_aeronave["AR"]
    e=dados_aeronave["e"]
    Cd0 = dados_aeronave["CD0"]
    BOW= dados_aeronave["BOW"]
    MTOW = dados_aeronave["MTOW"] # N
    Thrust0 = dados_aeronave["thrust"] # N
    S = dados_aeronave["A"]
    TSFC = dados_aeronave["TSFC"]
    K = 1/(np.pi*AR*e)

    W0 =  BOW + 31000 # N
    Thrust0 = 92300*0.02 # N thrust chute meu
    t_fa=0
    tempo_total=0
    tempo_total_fa=0
    consumo_total=0
    consumo_total_fa=0
    dist_percorrida_fa=0
    W=W0
    W_fa=W0

    T0 = Atmosphere(41000*0.3058).temperature
    a0 = Atmosphere(41000*0.3058).speed_of_sound
    P0 =  Atmosphere(41000*0.3058).pressure
    rho0 =  Atmosphere(41000*0.3058).density

    # Chutes do intervalo de velocidade a ser avaliado
    vmin = 50
    vmax = 100

    # Cria um range de velocidades a serem avaliadas
    V = np.arange(vmin, vmax+1, 10)

    # Variação da temperatura ISA
    dT = 15



    # variáveis auxiliares
    altitudes = []
    discretização=2000 # intervalos de altitudes
    dist_percorrida = 0
    t=0





    # realizando diversos cálculos para cada altura dentro do range de 0 à 40000 pés
    for h in np.arange(1,41001, discretização):
      h_real=41001-h

        # Corrige a variação de temperatura ISA
      T = Atmosphere(h_real*0.3048).temperature  # altitude em metros

      h_ind = h_real / (T + dT) *T



        #cálcul
      sigma = (Atmosphere(h_ind*0.3048).density)/rho0


        # variação do empuxo
      Thrust = Thrust0*sigma**0.7 # empuxo para menor q troposfera
      if h_real >36089:
        Thrust = 1.439*Thrust0*sigma # empuxo para altitudes maiores q troposfera


        ## Calcula Drag x V

      consumo=TSFC*Thrust*t #  com fator de aceleração



      consumo_total=consumo_total + consumo # com fator de aceleração


      W = W - consumo  # com fator de aceleração


      Cl = (2 * (W/S))/( Atmosphere(h_ind*0.3048).density * V**2)


      Cd = Cd0 + K * (Cl**2)

        ## Calcula Climb grad. x V

      E = Cl/Cd



      gamma = np.arcsin(Thrust/ W-1/E)


        ## Calculo da razão de descida

      RD = V*np.sin(gamma) # calculos de cl, cd, W e consumo de combustivel feitos com fator de aceleração.
       # com fator de aceleração






      index = np.where(RD == min(RD)) # pega o index do vetor RC em que RC é máximo
      v_rcmax = V[index] # pega a velocidade em que RC é máximo
      gamma_rcmax = gamma[index] # pega o gama em que RC é máximo
      v_horizontal = v_rcmax*np.cos(gamma_rcmax) #calcula a velocidade horizontal em que RC é máximo para determinada faixa de altitude


      t = discretização/ np.abs(min(RD)) # tempo para o maximo RC de cada altitude = faixa de altitude/RC
      tempo_total=tempo_total+t #tempo total de subida


      dist_percorrida = dist_percorrida + v_horizontal*t


    print(consumo_total)
descida_destino1()


[3918.05403546]


# Subida 2 p/ loiter

Calculo do trecho de subida 2 para cruseiro para aeroporto alternativo

In [ ]:
print('Subida 2 p/ loitter')
dados_aeronave['W_fuel'] = 27_000
a, consumo_total_subida_2, b, FL_2 = subida(dados_aeronave, condicoes['H_final'], condicoes['H_loiter'])

Subida 2 p/ loitter
h0 =  0


## Trecho em Espera

Implementação do trecho de loiter ou modificação de cruseiro para computar esse tre

In [ ]:
import ambiance as amb
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere


def tempo_de_espera():

    ## Dados da aeronave
    AR= dados_aeronave["AR"]
    e = dados_aeronave["e"]
    Cd0 = dados_aeronave["CD0"]

    MTOW = dados_aeronave["MTOW"] # N
    Thrust0 = dados_aeronave["thrust"] # N
    S = dados_aeronave["A"]
    TSFC = dados_aeronave["TSFC"]
    BOW = dados_aeronave["BOW"]
    K = 1/(np.pi*AR*e)



    # Condições atmosféricas
    isa = +15
    altitude = 10000*0.3058 # 10000 pés
    temp = Atmosphere(altitude).temperature - isa #Dados atmosféricos da biblioteca Ambiance
    a = Atmosphere(altitude).speed_of_sound #Dados atmosféricos da biblioteca Ambiance
    P = Atmosphere(altitude).pressure #Dados atmosféricos da biblioteca Ambiance
    rho =  Atmosphere(altitude).density


    # Peso total e de combustível ao se iniciar cruzeiro


    W =  BOW + 14100 + 9350 # N# Peso total no inicio da espera precisa mudar aqui
    Win = W


    # Cd e eficiência no início do cruzeiro


    dt = 10 # faixa de tempo de 100s para processo iterativo
    tempo = list(range(0,30*60,dt))



    for t in tempo:
      V_emax = np.sqrt(2*W/rho/S)*np.sqrt(np.sqrt(K/Cd0))
      Cl = (2 * (W/S))/( rho*V_emax**2)
      Cd = Cd0 + K * (Cl**2)
      E = Cl/Cd # Eficiência aerodinâmica
      T = W/E # corrigindo o empuxo
      W = W - TSFC*dt*T # peso da aeronave a cada faixa de tempo dt percorrida
    print('gasto total de combustível [N] =', Win-W)
tempo_de_espera()


gasto total de combustível [N] = [9347.86467201]


## Subida 3 p/ cruseiro 2

Calculo do trecho de subida 2 para cruseiro para aeroporto alternativo

In [ ]:
print('Subida 3 loitter p/ cruise 2')
dados_aeronave['W_fuel'] = 14_100
_, consumo_total_subida_3, _, FL_3 = subida(dados_aeronave, condicoes['H_loiter'], condicoes['H_cruise'])

Subida 3 loitter p/ cruise 2
h0 =  10000


## Cruseiro 2

Implementa o trecho de cruseiro 2

In [ ]:
import ambiance as amb
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere


def cruzeiro_alternativa():

    # Dados da aeronave
    AR= dados_aeronave["AR"]
    e = dados_aeronave["e"]
    Cd0 = dados_aeronave["CD0"]

    MTOW = dados_aeronave["MTOW"] # N
    Thrust0 = dados_aeronave["thrust"] # N
    S = dados_aeronave["A"]
    TSFC = dados_aeronave["TSFC"]
    MACH = 0.79
    K = 1/(np.pi*AR*e)

    # Condições atmosféricas
    isa = 0
    altitude = 20000*0.3058 # 20000 pés
    temp = Atmosphere(altitude).temperature - isa #Dados atmosféricos da biblioteca Ambiance
    a = Atmosphere(altitude).speed_of_sound #Dados atmosféricos da biblioteca Ambiance
    P = Atmosphere(altitude).pressure #Dados atmosféricos da biblioteca Ambiance
    rho =  Atmosphere(altitude).density
    dt=10
    distancia_total=0
    dist=0


    # Peso total e de combustível ao se iniciar cruzeiro
    W_finalRC = 325700 # precisa de 4000 pro cruzeiro e 1400 pro pouso
    W = W_finalRC # Peso total no inicio do cruzeiro
    tempo = list(range(0,2000000,dt))


    # Cd e eficiência no início do cruzeiro




    distancia_cruzeiro_total=0 #distancia total percorrida inicial
    dist_cruzeiro=0


    for t in tempo:
       v_operacional = MACH*a
       Cl = (2 * (W/S))/( Atmosphere(altitude).density * v_operacional**2) #calculo do cl
       Cd = Cd0 + K * (Cl**2)
       E = Cl/Cd
       T=W/E
       W = W - TSFC*dt*T
       dist = v_operacional * dt # distancia percorrida para cada faixa de tempo
       distancia_total = distancia_total + dist # soma de distancias percorridas para cada velocidade ideal

       if  distancia_total > 198000:
        print('A aeronave usou todo o combustível disponível para cruzeiro!\n')
        print('tempo total gasto em cruzeiro [min] = ', t/60)
        break


    print('ditancia total percorrida [km] =',distancia_total/1000)
    print('consumo de combustivel[N]=',W_finalRC - W)


cruzeiro_alternativa()


A aeronave usou todo o combustível disponível para cruzeiro!

tempo total gasto em cruzeiro [min] =  13.166666666666666
ditancia total percorrida [km] = [199.69525409]
consumo de combustivel[N]= [9304.68973937]


## Descida para pouso na alternativa

Implementa a descida para pouso

In [ ]:
import ambiance as amb
import numpy as np
import matplotlib.pyplot as plt
from ambiance import Atmosphere


def descida_alternativa():
    ## Dados da aeronave
   AR= dados_aeronave["AR"]
    e = dados_aeronave["e"]
    Cd0 = dados_aeronave["CD0"]
    g = condicoes["g"]
    MTOW = dados_aeronave["MTOW"] # N
    Thrust0 = dados_aeronave["thrust"] # N
    S = dados_aeronave["A"]
    TSFC = dados_aeronave["TSFC"]



    W0 = 321700 # N % chute teste, tentei com varios pesos iniciais. esse é oq chega mais proximo de acabar o combustivel no solo.
    Thrust0 = 92300*0.02 # N thrust chute meu


    t=0

    tempo_total=0

    consumo_total=0


    W=W0


    T0 = Atmosphere(20000*0.3058).temperature
    a0 = Atmosphere(20000*0.3058).speed_of_sound
    P0 =  Atmosphere(20000*0.3058).pressure
    rho0 =  Atmosphere(20000*0.3058).density

    # Chutes do intervalo de velocidade a ser avaliado
    vmin = 50
    vmax = 100

    # Cria um range de velocidades a serem avaliadas
    V = np.arange(vmin, vmax+1, 10)

    # Variação da temperatura ISA
    dT = +20



    # variáveis auxiliares
    altitudes = []
    discretização=1000 # intervalos de altitudes
    dist_percorrida = 0





    # realizando diversos cálculos para cada altura dentro do range de 0 à 40000 pés
    for h in np.arange(1,18001, discretização):
      h_real=20001-h

        # Corrige a variação de temperatura ISA
      T = Atmosphere(h_real*0.3048).temperature  # altitude em metros
      h_ind = h_real / (T + dT) *T

        #cálcul
      sigma = (Atmosphere(h_real*0.3048).density)/rho0



        # variação do empuxo
      Thrust = Thrust0*sigma**0.7 # empuxo para menor q troposfera

        ## Calcula Drag x V

      consumo=TSFC*Thrust*t #  com fator de aceleração



      consumo_total=consumo_total + consumo # com fator de aceleração


      W = W - consumo  # com fator de aceleração


      Cl = (2 * (W/S))/( Atmosphere(h_ind*0.3048).density * V**2)


      Cd = Cd0 + K * (Cl**2)

        ## Calcula Climb grad. x V

      E = Cl/Cd



      gamma = np.arcsin(Thrust/W - 1/E)


        ## Calculo da razão de descida

      RD= V*np.sin(gamma) # calculos de cl, cd, W e consumo de combustivel feitos com fator de aceleração.






      index = np.where(RD == min(RD)) # pega o index do vetor RC em que RC é máximo
      v_rcmax = V[index] # pega a velocidade em que RC é máximo
      gamma_rcmax = gamma[index] # pega o gama em que RC é máximo
      v_horizontal = v_rcmax*np.cos(gamma_rcmax) #calcula a velocidade horizontal em que RC é máximo para determinada faixa de altitude


      t = discretização/ np.abs(min(RD)) # tempo para o maximo RC de cada altitude = faixa de altitude/RC
      tempo_total=tempo_total+t #tempo total de subida


      dist_percorrida = dist_percorrida + v_horizontal*t
      print(consumo_total)


descida_alternativa()

IndentationError: ignored